## API Data Wrangling Mini-Project (completed)
This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv
from pandas.core.interchange import column

load_dotenv()
API_KEY = os.getenv('NASDAQ_DATA_LINK_API_KEY')

print(API_KEY[:4], (len(API_KEY)-8) * ".", API_KEY[-4:], sep="")

Azxo............AZf-


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

In [2]:
# !pip install requests

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
from collections import namedtuple, deque, ChainMap, Counter, OrderedDict, defaultdict

Note: API's can change a bit with each version, for this exercise it is recommended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [4]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X"
params = dict(api_key=API_KEY,
              limit=1)

res = requests.get(url, params)

print(res)

<Response [200]>


In [5]:
res.text

'{"dataset":{"id":10095370,"dataset_code":"AFX_X","database_code":"FSE","name":"Carl Zeiss Meditec (AFX_X)","description":"Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.\\u003cbr\\u003e\\u003cbr\\u003eTrading System: Xetra\\u003cbr\\u003e\\u003cbr\\u003eISIN: DE0005313704","refreshed_at":"2020-12-01T14:48:09.907Z","newest_available_date":"2020-12-01","oldest_available_date":"2000-06-07","column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"frequency":"daily","type":"Time Series","premium":false,"limit":1,"transform":null,"column_index":null,"start_date":"2000-06-07","end_date":"2020-12-01","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,51.0,5703.0,null,null,null]],"collapse":null,"order":null,"database_id":6129}}'

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
res.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': 1,
  'transform': None,
  'column_index': None,
  'start_date': '2000-06-07',
  'end_date': '2020-12-01',
  'data': [['2020-12-01',
    112.2,
    112.2,
    111.5,
    112.0,
    None,
    51.0,
    5703.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

In [7]:
params = dict(api_key=API_KEY,
              start_date="2017-01-01",
              end_date="2017-12-31")

res = requests.get(url, params)
res.json().keys()

dict_keys(['dataset'])

#### 2. Convert the returned JSON object into a Python dictionary.

In [8]:
all2017 = res.json()['dataset']
all2017.keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [9]:
print(all2017['column_names'])
print(all2017['data'][0])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [10]:
clean_column_names = [s.replace(" ", "_").lower() for s in all2017['column_names']]
Datum = namedtuple('Datum', clean_column_names)
data = [Datum(*d) for d in sorted(all2017['data'], key=lambda x: x[0])]  # Sort by date
data[0]

Datum(date='2017-01-02', open=34.99, high=35.94, low=34.99, close=35.8, change=None, traded_volume=44700.0, turnover=1590561.0, last_price_of_the_day=None, daily_traded_units=None, daily_turnover=None)

In [11]:
data[-1]

Datum(date='2017-12-29', open=51.76, high=51.94, low=51.45, close=51.76, change=None, traded_volume=34640.0, turnover=1792304.0, last_price_of_the_day=None, daily_traded_units=None, daily_turnover=None)

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [12]:
opens = [d.open for d in data if d.open]
print(*opens[:5], "...", *opens[-5:])

34.99 35.9 35.48 35.02 34.91 ... 51.16 51.05 51.45 51.65 51.76


In [13]:
sorted_by_open = sorted([d for d in data if d.open], key=lambda x: x.open)
highest = sorted_by_open[-1]
lowest = sorted_by_open[0]
print(f"In 2017, the highest opening price for AFX_X was €{highest.open:.2f} (on {highest.date}),")
print(f"and the lowest opening price was €{lowest.open:.2f} (on {lowest.date}).")

In 2017, the highest opening price for AFX_X was €53.11 (on 2017-12-14),
and the lowest opening price was €34.00 (on 2017-01-24).


#### 4. What was the largest change in any one day (based on High and Low price)?

In [14]:
sorted_by_range = sorted([d for d in data if d.low and d.high], key=lambda x: x.high-x.low)
largest = sorted_by_range[-1]
print(f"The largest range of prices in any one day was "
      f"€{largest.high - largest.low:.2f} (on {largest.date}).")

The largest range of prices in any one day was €2.81 (on 2017-05-11).


#### 5. What was the largest change between any two days (based on Closing Price)?

In [15]:
i_max = 0
change_max = 0
for i, d in enumerate(data[:-1]):
    change = data[i+1].close - d.close
    if abs(change) > abs(change_max):
        change_max = change
        i_max = i+1
        
print(f"Largest day-over-day change of closing prices was €{change_max:.2f}"
      f" (between {data[i_max].date} and {data[i_max+1].date}).")

Largest day-over-day change of closing prices was €-2.56 (between 2017-08-09 and 2017-08-10).


#### 6. What was the average daily trading volume during this year?

In [16]:
volumes = [d.traded_volume for d in data]
print(*volumes[:5], "...", *volumes[-5:])  # head & tail

44700.0 70618.0 54408.0 48412.0 27507.0 ... 120649.0 71165.0 57452.0 40660.0 34640.0


In [17]:
print(f"Trading volume averaged {sum(volumes) / len(volumes):.0f} trades per day.")

Trading volume averaged 89124 trades per day.


#### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [18]:
def median(numbers):
    nn = sorted(numbers)
    i = len(nn)//2
    if len(nn)%2 != 0:  # Odd number of numbers.
        return nn[i]
    else:  # Even number of numbers.
        return (nn[i-1] + nn[i]) / 2
    
print(f"Median trading volume was {median(volumes):.0f} trades per day.")

Median trading volume was 76286 trades per day.
